In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/asap-aes/training_set_rel3.tsv


# Initial Setup

In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GEMINI_API_KEY")
secret_value_1 = user_secrets.get_secret("HUGGINGFACE_TOKEN")

## Download all the files

In [4]:
!rm -rf /kaggle/working/*

In [3]:
import zipfile
import os
from IPython.display import FileLink

def bundle_all_outputs(zip_name='full_project_outputs'):
    zip_path = f"/kaggle/working/{zip_name}.zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk('/kaggle/working/'):
            for file in files:
                if not file.endswith('.zip'):  # Avoid nesting zips
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '/kaggle/working/')
                    zipf.write(file_path, arcname)
    display(FileLink(zip_path))
    print(f"✓ Full bundle ready: {zip_path}")

# Run this to zip and get download link
bundle_all_outputs()

/kaggle/working/full_project_outputs.zip

✓ Full bundle ready: /kaggle/working/full_project_outputs.zip


# Methodology

## Gemini Integration

In [29]:
!pip install -q google-generativeai
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GEMINI_API_KEY")
genai.configure(api_key=api_key)
print("✓ Gemini API configured")

✓ Gemini API configured


In [6]:
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    generation_config={"temperature": 0.3, "top_p": 0.8, "max_output_tokens": 200},
    safety_settings=safety_settings
)
print("✓ Model initialized with safety and config")

✓ Model initialized with safety and config


## Loading asap-aes dataset

In [14]:
asap_path = Path('/kaggle/input/asap-aes')
training_file = asap_path / 'training_set_rel3.tsv'
df_asap = pd.read_csv(training_file, sep='\t', encoding='ISO-8859-1')

df_asap

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12969,21626,8,In most stories mothers and daughters are eit...,17,18,NaN,35,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12970,21628,8,I never understood the meaning laughter is th...,15,17,NaN,32,NaN,NaN,NaN,...,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
12971,21629,8,"When you laugh, is @CAPS5 out of habit, or is ...",20,26,40.0,40,NaN,NaN,NaN,...,5.0,5.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,4.0
12972,21630,8,Trippin' on fen...,20,20,NaN,40,NaN,NaN,NaN,...,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
df_asap.isnull().sum()

essay_id              0
essay_set             0
essay                 0
rater1_domain1        0
rater2_domain1        0
rater3_domain1    12846
domain1_score         0
rater1_domain2    11174
rater2_domain2    11174
domain2_score     11174
rater1_trait1     10682
rater1_trait2     10682
rater1_trait3     10682
rater1_trait4     10682
rater1_trait5     12251
rater1_trait6     12251
rater2_trait1     10682
rater2_trait2     10682
rater2_trait3     10682
rater2_trait4     10682
rater2_trait5     12251
rater2_trait6     12251
rater3_trait1     12846
rater3_trait2     12846
rater3_trait3     12846
rater3_trait4     12846
rater3_trait5     12846
rater3_trait6     12846
dtype: int64

## Cleaning the dataset

In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
import time  # For rate limiting

# Load ASAP-AES (standard paths in /kaggle/input/asap-aes/)
asap_path = Path('/kaggle/input/asap-aes')
training_file = asap_path / 'training_set_rel3.tsv'
df_asap = pd.read_csv(training_file, sep='\t', encoding='ISO-8859-1')

# Define prompt mapping for each essay_set (1-8, from ASAP++ paper)
prompt_dict = {
    1: "The writers had to write a letter to their local newspaper in which they stated their opinion on the effects computers have on people.",
    2: "The writers had to write a persuasive essay reflecting their views on censorship in libraries.",
    3: "The writers had to read an extract from *Rough Road Ahead: Do Not Exceed Posted Speed Limit* by Joe Kurmaskie. They then had to explain how the features of the setting affected the cyclist.",
    4: "The writers had to read an extract from *Winter Hibiscus* by Minfong Ho. They then had to explain why the author concludes the story in the way that she did.",
    5: "The writers had to read an extract from *Narciso Rodriguez* by Narciso Rodriguez. They then had to describe the mood created by the author with supporting evidence from the extract.",
    6: "The writers had to read an extract from *The Mooring Mast* by Marcia Amidon Lusted. They then had to answer a question about the difficulties faced by the builders of the Empire State Building in allowing dirigibles to dock there.",
    7: "Write a story about a time when you, or someone you know, was patient.",
    8: "Write a story in which laughter plays a part."
}

# Clean and sample 500 rows (focus on essay_set and essays)
df_asap = df_asap.dropna(subset=['essay_set', 'essay'])
df_sample = df_asap.sample(n=500, random_state=42).reset_index(drop=True)

# Assign prompts based on essay_set
df_sample['prompt'] = df_sample['essay_set'].map(prompt_dict)
df_sample['prompt'] = df_sample['prompt'].fillna("Generic essay prompt for Bloom's classification.")  # Rare fallback

df_sample['response'] = df_sample['essay']  # Student response
df_sample['essay_score'] = df_sample['domain1_score']  # Use primary holistic score
df_sample = df_sample[['prompt', 'response', 'essay_score', 'essay_set']]  # Unified format base + essay_set for reference

print(f"✓ Loaded {len(df_sample)} ASAP samples with mapped prompts")
save_checkpoint("asap_sample", df_sample, "Raw ASAP sample for labeling")

✓ Loaded 500 ASAP samples with mapped prompts
✓ SAVED (root): asap_sample (Raw ASAP sample for labeling)


'/kaggle/working/checkpoint_asap_sample_20251103_184619'

## Bloom's Taxonomy Dataset Integration

In [18]:
pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 5.3 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [20]:
"""
DOCX to DataFrame Converter for Bloom's Taxonomy Dataset

This script converts a structured DOCX file containing questions 
organized by Bloom's taxonomy levels into a pandas DataFrame.

Structure expected:
- Bloom's level headers: Knowledge, Comprehension, Application, Analysis, Synthesis, Evaluation
- Questions listed under each header
"""

import docx
import pandas as pd
from pathlib import Path


def convert_blooms_docx_to_dataframe(docx_path, output_csv=None):
    """
    Convert a Bloom's taxonomy structured DOCX to a pandas DataFrame.
    
    Parameters:
    -----------
    docx_path : str or Path
        Path to the DOCX file
    output_csv : str or Path, optional
        If provided, saves the DataFrame to this CSV path
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns: ['bloom_level', 'question', 'question_id']
    """
    
    # Load the document
    doc = docx.Document(docx_path)
    
    # Define Bloom's taxonomy levels (in order)
    bloom_levels = ['Knowledge', 'Comprehension', 'Application', 
                    'Analysis', 'Synthesis', 'Evaluation']
    
    # Storage for parsed data
    data = []
    current_level = None
    question_counter = {level: 0 for level in bloom_levels}
    
    # Parse document
    for para in doc.paragraphs:
        text = para.text.strip()
        
        # Skip empty paragraphs
        if not text:
            continue
        
        # Check if this is a Bloom's level header
        if text in bloom_levels:
            current_level = text
            print(f"Found Bloom's level: {current_level}")
            continue
        
        # If we have a current level, this is a question
        if current_level:
            question_counter[current_level] += 1
            question_id = f"{current_level[:3].upper()}_{question_counter[current_level]:03d}"
            
            data.append({
                'question_id': question_id,
                'bloom_level': current_level,
                'question': text
            })
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    # Print summary statistics
    print("\n" + "="*60)
    print("CONVERSION SUMMARY")
    print("="*60)
    print(f"Total questions extracted: {len(df)}")
    print("\nQuestions per Bloom's level:")
    print(df['bloom_level'].value_counts().sort_index())
    
    # Save to CSV if requested
    if output_csv:
        df.to_csv(output_csv, index=False)
        print(f"\n✓ Saved to: {output_csv}")
    
    return df


def validate_dataframe(df):
    """
    Perform basic validation checks on the DataFrame.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The DataFrame to validate
        
    Returns:
    --------
    dict : Validation results
    """
    validation = {
        'total_questions': len(df),
        'missing_values': df.isnull().sum().to_dict(),
        'unique_levels': df['bloom_level'].nunique(),
        'expected_levels': 6,
        'duplicate_questions': df['question'].duplicated().sum(),
        'empty_questions': (df['question'].str.strip() == '').sum(),
        'avg_question_length': df['question'].str.len().mean(),
        'min_question_length': df['question'].str.len().min(),
        'max_question_length': df['question'].str.len().max()
    }
    
    print("\n" + "="*60)
    print("DATA QUALITY CHECKS")
    print("="*60)
    print(f"Total questions: {validation['total_questions']}")
    print(f"Unique Bloom's levels: {validation['unique_levels']}/{validation['expected_levels']}")
    print(f"Duplicate questions: {validation['duplicate_questions']}")
    print(f"Empty questions: {validation['empty_questions']}")
    print(f"Average question length: {validation['avg_question_length']:.1f} characters")
    print(f"Question length range: {validation['min_question_length']} - {validation['max_question_length']}")
    
    return validation


def preview_samples(df, n=3):
    """
    Display sample questions from each Bloom's level.
    
    Parameters:
    -----------
    df : pd.DataFrame
        The DataFrame to sample from
    n : int
        Number of samples per level
    """
    print("\n" + "="*60)
    print("SAMPLE QUESTIONS (3 per level)")
    print("="*60)
    
    for level in df['bloom_level'].unique():
        print(f"\n{level.upper()}:")
        samples = df[df['bloom_level'] == level].head(n)
        for idx, row in samples.iterrows():
            print(f"  [{row['question_id']}] {row['question'][:100]}...")


if __name__ == "__main__":
    # File paths
    input_docx = "/kaggle/input/researchgate-blooms-tax/BCLsDataSet.docx"
    output_csv = "/kaggle/working/bcls.csv"
    
    # Convert DOCX to DataFrame
    df = convert_blooms_docx_to_dataframe(input_docx, output_csv)
    
    # Validate the data
    validation_results = validate_dataframe(df)
    
    # Show samples
    preview_samples(df, n=3)
    
    # Display first few rows
    print("\n" + "="*60)
    print("FIRST 10 ROWS")
    print("="*60)
    print(df.head(10).to_string())
    
    print("\n✓ Conversion complete!")

Found Bloom's level: Knowledge
Found Bloom's level: Comprehension
Found Bloom's level: Application
Found Bloom's level: Analysis
Found Bloom's level: Synthesis
Found Bloom's level: Evaluation

CONVERSION SUMMARY
Total questions extracted: 600

Questions per Bloom's level:
bloom_level
Analysis         100
Application      100
Comprehension    100
Evaluation       100
Knowledge        100
Synthesis        100
Name: count, dtype: int64

✓ Saved to: /kaggle/working/bcls.csv

DATA QUALITY CHECKS
Total questions: 600
Unique Bloom's levels: 6/6
Duplicate questions: 1
Empty questions: 0
Average question length: 65.0 characters
Question length range: 16 - 247

SAMPLE QUESTIONS (3 per level)

KNOWLEDGE:
  [KNO_001] About what proportion of the population of the US is living on farms?...
  [KNO_002] Correctly label the brain lobes indicated on the diagram below...
  [KNO_003] Define compound interest....

COMPREHENSION:
  [COM_001] Compare Calliope with Howie. Use the word bank....
  [COM_002] Co

In [21]:
df_sample.head()

,prompt,response,essay_score,essay_set
0,The writers had to read an extract from *Winte...,The author concludes the story with this parag...,2,4
1,The writers had to read an extract from *Winte...,I think the author concludes the story with th...,2,4
2,The writers had to write a letter to their loc...,"Computers, a very much talked about subject. D...",10,1
3,"Write a story about a time when you, or someon...",One day when I was patient is when I was waiti...,17,7
4,The writers had to read an extract from *Rough...,The setting that affect the cyclist is the con...,1,3


## Data Labelling

In [44]:
# -------------------------------------------------
# 1. Imports & Gemini setup
# -------------------------------------------------
import pandas as pd
import json
import time
from tqdm import tqdm
import google.generativeai as genai
from pathlib import Path
from sklearn.model_selection import train_test_split

# (your GEMINI_API_KEY already loaded)
genai.configure(api_key=UserSecretsClient().get_secret("GEMINI_API_KEY"))

# -------------------------------------------------
# 2. Load Bloom's taxonomy examples
# -------------------------------------------------
blooms_path = Path('/kaggle/input/bloom-levels/bcls.csv')  # adjust if needed
df_blooms = pd.read_csv(blooms_path)

# Map original → Revised Bloom
level_map = {
    'Knowledge': 'Remember',
    'Comprehension': 'Understand',
    'Application': 'Apply',
    'Analysis': 'Analyze',
    'Synthesis': 'Create',
    'Evaluation': 'Evaluate'
}
df_blooms['bloom_level'] = df_blooms['bloom_level'].map(level_map)

# -------------------------------------------------
# 3. Build few-shot examples (3 per level)
# -------------------------------------------------
samples_per_level = 3
df_fewshot = (
    df_blooms.groupby('bloom_level')
    .apply(lambda g: g.sample(n=min(len(g), samples_per_level), random_state=42))
    .reset_index(drop=True)
)

def build_fewshot_text(df):
    lines = []
    for _, r in df.iterrows():
        lines.append(
            f"Question: {r['question']}\n"
            f"Bloom Level: {r['bloom_level']}\n"
            f"Score: 5\n"
            f"Rationale: Matches {r['bloom_level']} verbs."
        )
    return "\n\n".join(lines)

fewshot_text = build_fewshot_text(df_fewshot)

# -------------------------------------------------
# 4. Prompt template (only the *prompt* is inserted)
# -------------------------------------------------
labeling_prompt_template = (
    "You are a linguistics expert analyzing the cognitive demand of a question.\n"
    "Assign **one** of the six Revised Bloom levels (Remember, Understand, Apply, Analyze, Evaluate, Create).\n\n"
    f"Examples:\n{fewshot_text}\n\n"
    "Question: {prompt}\n\n"
    "Respond **only** with valid JSON:\n"
    "{{\n"
    "  \"bloom_level\": \"Level\",\n"
    "  \"relevance\": 1-5,\n"
    "  \"rationale\": \"Brief reason (1-2 sentences)\"\n"
    "}}"
)

# -------------------------------------------------
# 5. Gemini model (free-tier correct name from your dashboard)
# -------------------------------------------------
model = genai.GenerativeModel(
    model_name="gemini-2.5-flash-lite",  
    generation_config={
        "temperature": 0.3,
        "top_p": 0.8,
        "max_output_tokens": 200
    }
)

# -------------------------------------------------
# 6. Labeling function (batch + progress + checkpoints)
# -------------------------------------------------
def label_with_gemini(df, batch_size=10, checkpoint_every=100):
    results = []
    for start in tqdm(range(0, len(df), batch_size), desc="Labeling Batches"):
        batch = df.iloc[start:start+batch_size]
        for _, row in batch.iterrows():
            prompt = labeling_prompt_template.format(prompt=row['prompt'])
            try:
                resp = model.generate_content(prompt)
                # Safety: guard empty response
                if not resp.candidates or not resp.candidates[0].content.parts:
                    raise ValueError("Empty response")
                txt = resp.candidates[0].content.parts[0].text.strip()
                txt = txt.replace('```json', '').replace('```', '').strip()
                data = json.loads(txt)

                row_labeled = row.copy()
                row_labeled['bloom_level'] = data.get('bloom_level', 'Unknown')
                row_labeled['label_score'] = data.get('relevance', 0)
                row_labeled['rationale'] = data.get('rationale', 'Failed')
                results.append(row_labeled)

            except Exception as e:
                print(f"\nError on row {row.name}: {e}")
                row_labeled = row.copy()
                row_labeled['bloom_level'] = 'Unknown'
                row_labeled['label_score'] = 0
                row_labeled['rationale'] = 'Labeling failed'
                results.append(row_labeled)

            time.sleep(5)  # 15 RPM → ~1 request / 4s (safe buffer)

        # ---- partial checkpoint ----
        if (start + batch_size) % checkpoint_every == 0 and results:
            pd.DataFrame(results).to_csv(
                f"/kaggle/working/checkpoint_labeled_{start+len(batch)}.csv",
                index=False
            )
            print(f"  partial save: {len(results)} rows")

    return pd.DataFrame(results)

# -------------------------------------------------
# 7. RUN on the full df_sample
# -------------------------------------------------
df_labeled = label_with_gemini(df_sample, batch_size=10, checkpoint_every=100)

# -------------------------------------------------
# 8. Final save + split
# -------------------------------------------------
save_checkpoint("final_labeled_500", df_labeled, "Full Bloom-labeled ASAP (prompt-only)")

train_val, test = train_test_split(
    df_labeled, test_size=0.2, random_state=42, stratify=df_labeled['essay_set']
)
train, val = train_test_split(
    train_val, test_size=0.25, random_state=42, stratify=train_val['essay_set']
)

save_checkpoint(
    "splits",
    {'train': train, 'val': val, 'test': test},
    "60/20/20 split, stratified by essay_set"
)

# -------------------------------------------------
# 9. Quick sanity check
# -------------------------------------------------
print("\nFirst 5 labeled rows:")
print(df_labeled[['prompt', 'bloom_level', 'label_score', 'rationale']].head())
print("\nLevel distribution:")
print(df_labeled['bloom_level'].value_counts())

/tmp/ipykernel_37/17751501.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min(len(g), samples_per_level), random_state=42))
Labeling Batches:  20%|██        | 10/50 [09:21<37:26, 56.17s/it]

  partial save: 100 rows


Labeling Batches:  40%|████      | 20/50 [18:43<28:08, 56.29s/it]

  partial save: 200 rows


Labeling Batches:  60%|██████    | 30/50 [28:08<18:51, 56.57s/it]

  partial save: 300 rows


Labeling Batches:  80%|████████  | 40/50 [37:30<09:21, 56.14s/it]

  partial save: 400 rows


Labeling Batches: 100%|██████████| 50/50 [46:51<00:00, 56.23s/it]

  partial save: 500 rows
✓ SAVED (root): final_labeled_500 (Full Bloom-labeled ASAP (prompt-only))
✓ SAVED (root): splits (60/20/20 split, stratified by essay_set)

First 5 labeled rows:
                                              prompt bloom_level  label_score  \
0  The writers had to read an extract from *Winte...     Analyze            5   
1  The writers had to read an extract from *Winte...     Analyze            5   
2  The writers had to write a letter to their loc...      Create            5   
3  Write a story about a time when you, or someon...      Create            5   
4  The writers had to read an extract from *Rough...     Analyze            5   

                                           rationale  
0  The question requires students to break down t...  
1  The question requires students to break down t...  
2  The question requires students to produce a ne...  
3  The question asks the user to 'write a story,'...  
4  The question requires the writer to break down..

In [3]:
# Revised DIRS for flatter structure
DIRS = {
    'models': '/kaggle/working/models',      # Keep for large files
    'data': '/kaggle/working/data',         # Keep for inputs/processed
    'prompts': '/kaggle/working/prompts',   # Keep for templates
    'gold_labels': '/kaggle/working/gold_labels',  # Keep for evals
    # Flatten these:
    # 'checkpoints': '/kaggle/working/checkpoints',  # → Use root
    # 'results': '/kaggle/working/results',         # → Use root
}

# # Create dirs (skip flattened ones)
# for dir_path in DIRS.values():
#     Path(dir_path).mkdir(parents=True, exist_ok=True)

# Update save_checkpoint to root
def save_checkpoint(name: str, data, description: str = "", metadata: Dict = None):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    base_path = f"/kaggle/working/checkpoint_{name}_{timestamp}"  # Flattened!
    
    # Rest unchanged...
    with open(f"{base_path}.pkl", 'wb') as f:
        pickle.dump(data, f)
    
    meta = { 'timestamp': timestamp, 'description': description, 'data_type': type(data).__name__, **(metadata or {}) }
    with open(f"{base_path}_meta.json", 'w') as f:
        json.dump(meta, f, indent=2)
    
    print(f"✓ SAVED (root): {name} ({description})")
    return base_path

# New: Auto-zip all root outputs at end
def bundle_all_outputs(zip_name='full_project_outputs'):
    zip_path = f"/kaggle/working/{zip_name}.zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk('/kaggle/working/'):
            for file in files:
                if not file.endswith('.zip'):  # Avoid nesting zips
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '/kaggle/working/')
                    zipf.write(file_path, arcname)
    display(FileLink(zip_path))
    print(f"✓ Full bundle ready: {zip_path}")

NameError: name 'Dict' is not defined

In [1]:
# Usage: Call at notebook end
bundle_all_outputs()

NameError: name 'bundle_all_outputs' is not defined

## Analysis

It turns out that the dataset ASAP (essays of students) consists of prompts that are either 'create' or 'analyze' ONLY.

# Fine Tuning with LoRA

In [2]:
!pip install -q --force-reinstall "numpy<2" pyarrow==14.0.1 datasets==2.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 82.0 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
pathos 0.3.2 requires dill>=0.3.8, but you have dill 0.3.7 which is incompatible.
pathos 0.3.2 requires multiprocess>=0.70.16, but you have multiprocess 0.70.15 which is in

In [ ]:
import os
os._exit(00)

In [1]:
# -------------------------------------------------
# 1. Install & Imports
# -------------------------------------------------
!pip install -q transformers peft bitsandbytes accelerate datasets sentencepiece

import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import os

# -------------------------------------------------
# 2. Load Labeled Train Data
# -------------------------------------------------
# Load the saved splits dict
splits = pd.read_pickle("/kaggle/working/checkpoint_splits_20251103_211458.pkl")  # or your path

# Extract the train DataFrame
train_df = splits['train']
val_df = splits['val']
test_df = splits['test']

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# -------------------------------------------------
# 3. Format Instruction Pairs (Bloom in system prompt)
# -------------------------------------------------
def format_instruction(row):
    system = f"You are an expert evaluator at the {row['bloom_level']} level of Bloom's Taxonomy."
    user = f"Prompt: {row['prompt']}\n\nStudent Response: {row['response']}\n\nScore this response from 1 to 5 and provide a brief rationale."
    return {
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ]
    }

train_data = [format_instruction(row) for _, row in train_df.iterrows()]
hf_dataset = Dataset.from_list(train_data)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 32.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 41.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 3.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

2025-11-03 23:03:49.615667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762211029.639411     127 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762211029.646311     127 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Train: 300, Val: 100, Test: 100


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("GEMINI_API_KEY")
secret_value_1 = user_secrets.get_secret("HUGGINGFACE_TOKEN")
secret_value_2 = user_secrets.get_secret("WANDB_API_KEY")

In [10]:
# -------------------------------------------------
# 1. Install & Setup W&B
# -------------------------------------------------
!pip install -q wandb

In [2]:
import wandb
from kaggle_secrets import UserSecretsClient

# Get W&B key from Kaggle secrets
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

# Login programmatically (no prompt)
wandb.login(key=wandb_api_key)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

True

In [ ]:
# -------------------------------------------------
# OPTIMIZED FOR P100 - Run after switching
# -------------------------------------------------

import os
# No need for CUDA_VISIBLE_DEVICES with single GPU
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

import torch
import gc
import pandas as pd
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    TrainingArguments, 
    Trainer, 
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset
import wandb

# Verify P100
print("="*60)
print("GPU CHECK")
print("="*60)
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    total_mem = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"✓ GPU: {gpu_name}")
    print(f"✓ Memory: {total_mem:.2f} GB")
    
    if "P100" not in gpu_name:
        print("⚠️ WARNING: Not using P100! Switch accelerator.")
    else:
        print("✓ Perfect! P100 is ideal for this task.")
else:
    print("✗ No GPU found!")
print("="*60)

# Load data
import pickle
with open("/kaggle/working/checkpoint_splits_20251103_211458.pkl", "rb") as f:
    splits = pickle.load(f)

train_df = splits['train']
val_df = splits['val']
test_df = splits['test']

def format_instruction(row):
    system = f"You are an expert evaluator at the {row['bloom_level']} level of Bloom's Taxonomy."
    user = f"Prompt: {row['prompt']}\n\nStudent Response: {row['response']}\n\nScore this response from 1 to 5 and provide a brief rationale."
    return {
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": user}
        ]
    }

train_data = [format_instruction(row) for _, row in train_df.iterrows()]
hf_dataset = Dataset.from_list(train_data)

print(f"✓ Dataset: {len(hf_dataset)} examples")

# -------------------------------------------------
# Training Function - Optimized for P100
# -------------------------------------------------

def finetune_model_p100(model_id, output_dir):
    """Training optimized for single P100 GPU"""
    
    print(f"\n{'='*60}")
    print(f"Training: {model_id}")
    print(f"{'='*60}")
    
    os.makedirs(output_dir, exist_ok=True)
    gc.collect()
    torch.cuda.empty_cache()
    
    # W&B
    run = wandb.init(
        project="llm-jury-bloom-p100",
        name=f"{model_id.split('/')[-1]}-lora",
        config={
            "model": model_id,
            "gpu": "P100",
            "train_size": len(train_df),
            "lora_r": 8,
            "lora_alpha": 16,
            "batch_size": 4,  # Can use 4 on P100!
            "grad_accum": 4,
            "steps": 300,
            "max_length": 512
        }
    )
    
    try:
        # Quantization
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4"
        )
        
        # Load model
        print("Loading model...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            quantization_config=bnb_config,
            device_map="auto",
            torch_dtype=torch.bfloat16,
            trust_remote_code=True,
            use_cache=False
        )
        
        tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "right"
        
        # LoRA
        print("Applying LoRA...")
        model.gradient_checkpointing_enable()
        model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)
        
        lora_config = LoraConfig(
            r=8,
            lora_alpha=16,
            target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
            lora_dropout=0.05,
            bias="none",
            task_type="CAUSAL_LM"
        )
        model = get_peft_model(model, lora_config)
        model.print_trainable_parameters()
        
        # Tokenize
        def tokenize_fn(examples):
            texts = []
            for msgs in examples["messages"]:
                text = ""
                for msg in msgs:
                    text += f"{msg['role']}: {msg['content']}\n"
                texts.append(text)
            
            result = tokenizer(
                texts,
                truncation=True,
                max_length=512,
                padding="max_length",
                return_tensors=None
            )
            result["labels"] = result["input_ids"].copy()
            return result
        
        print("Tokenizing...")
        tokenized = hf_dataset.map(
            tokenize_fn,
            batched=True,
            remove_columns=hf_dataset.column_names
        )
        
        # Training args - Better for P100
        args = TrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=4,  # Increased from 2!
            gradient_accumulation_steps=4,  # Decreased from 8
            max_steps=300,
            learning_rate=2e-4,
            fp16=True,
            logging_steps=10,
            save_steps=150,
            save_total_limit=1,
            warmup_steps=10,
            weight_decay=0.01,
            report_to="wandb",
            gradient_checkpointing=True,
            optim="paged_adamw_8bit"
        )
        
        trainer = Trainer(model=model, args=args, train_dataset=tokenized)
        
        print("\nTraining...")
        trainer.train()
        
        print(f"\nSaving to {output_dir}...")
        model.save_pretrained(output_dir)
        tokenizer.save_pretrained(output_dir)
        
        print("✓ Complete!")
        return True
        
    except Exception as e:
        print(f"✗ Error: {e}")
        import traceback
        traceback.print_exc()
        return False
        
    finally:
        wandb.finish()
        if 'model' in locals():
            del model
        if 'trainer' in locals():
            del trainer
        gc.collect()
        torch.cuda.empty_cache()

# -------------------------------------------------
# Train All 3 Models
# -------------------------------------------------

models = [
    ("microsoft/Phi-3-mini-4k-instruct", "/kaggle/working/models/phi3_lora"),
    ("mistralai/Mistral-7B-Instruct-v0.3", "/kaggle/working/models/mistral_lora"),
    ("Qwen/Qwen2.5-7B-Instruct", "/kaggle/working/models/qwen_lora"),
]

print("\n🚀 Training on P100 (16GB)")
print("Batch size: 4, Grad accumulation: 4 (effective batch = 16)")

results = {}
for model_id, out_dir in models:
    print(f"\n{'#'*60}")
    print(f"Model: {model_id}")
    print(f"{'#'*60}")
    
    success = finetune_model_p100(model_id, out_dir)
    results[model_id] = success

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
for model, success in results.items():
    print(f"{'✓' if success else '✗'} {model}")

GPU CHECK
✓ GPU: Tesla P100-PCIE-16GB
✓ Memory: 15.89 GB
✓ Perfect! P100 is ideal for this task.
✓ Dataset: 300 examples

🚀 Training on P100 (16GB)
Batch size: 4, Grad accumulation: 4 (effective batch = 16)

############################################################
Model: microsoft/Phi-3-mini-4k-instruct
############################################################

Training: microsoft/Phi-3-mini-4k-instruct


Loading model...


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

Applying LoRA...
trainable params: 5,046,272 || all params: 7,620,662,784 || trainable%: 0.0662
Tokenizing...


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



Training...


Step,Training Loss
10,7.077900
20,1.983200
30,1.535100
40,1.364500
50,1.303600
60,1.316900
70,1.286000
80,1.190700
90,1.158600
100,1.151300


In [59]:
import torch, gc
torch.cuda.empty_cache()
gc.collect()

0

In [4]:
# -------------------------------------------------
# PART 1: Diagnostic - What's Using Your GPU?
# -------------------------------------------------

import torch
import gc
import subprocess
import os

def check_gpu_processes():
    """See what's actually running on your GPU"""
    print("\n" + "="*60)
    print("GPU PROCESS CHECK")
    print("="*60)
    
    try:
        result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
        print(result.stdout)
    except:
        print("nvidia-smi not available")
    
    if torch.cuda.is_available():
        print("\nPyTorch GPU Memory:")
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            total = torch.cuda.get_device_properties(i).total_memory / 1024**3
            free = total - allocated
            print(f"GPU {i}:")
            print(f"  Total: {total:.2f} GB")
            print(f"  Allocated: {allocated:.2f} GB")
            print(f"  Reserved: {reserved:.2f} GB")
            print(f"  Free: {free:.2f} GB")

def list_gpu_tensors():
    """Find all tensors currently on GPU"""
    print("\n" + "="*60)
    print("TENSORS ON GPU")
    print("="*60)
    
    import sys
    tensor_count = 0
    total_size = 0
    
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj):
                if obj.is_cuda:
                    tensor_count += 1
                    total_size += obj.element_size() * obj.nelement()
                    if tensor_count <= 10:  # Show first 10
                        print(f"  Tensor: {obj.size()}, {obj.dtype}, {obj.element_size() * obj.nelement() / 1024**2:.2f} MB")
        except:
            pass
    
    print(f"\nTotal GPU tensors: {tensor_count}")
    print(f"Total size: {total_size / 1024**3:.2f} GB")

# Run diagnostics
check_gpu_processes()
list_gpu_tensors()


GPU PROCESS CHECK
Mon Nov  3 22:51:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P0             30W /   70W |   11591MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+----------------------------

/usr/local/lib/python3.11/dist-packages/torch/__init__.py:1113: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)



Total GPU tensors: 636
Total size: 8.02 GB


In [ ]:
# This will restart the kernel
import os
os._exit(0)

In [12]:
# -------------------------------------------------
# FORCE USE OF GPU 1 (the clean one)
# -------------------------------------------------
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Use only GPU 1

# Verify it worked
import torch
print(f"Using GPU: {torch.cuda.get_device_name(0)}")
print(f"Available memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"Currently allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")

# Now proceed with training

Using GPU: Tesla T4
Available memory: 14.74 GB
Currently allocated: 7.48 GB


In [5]:
!pip install -q transformers peft bitsandbytes wandb sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━

In [7]:
!pip install -U bitsandbytes

# Loading Adapters

In [1]:
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

def load_lora(base_id, lora_path):
    print(f"Loading {lora_path}...")
    base = AutoModelForCausalLM.from_pretrained(
        base_id, load_in_4bit=True, device_map="auto", torch_dtype=torch.bfloat16
    )
    model = PeftModel.from_pretrained(base, lora_path)
    tokenizer = AutoTokenizer.from_pretrained(base_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# Load all
phi, phi_tok = load_lora("microsoft/Phi-3-mini-4k-instruct", "models/phi3_lora")
mistral, mistral_tok = load_lora("mistralai/Mistral-7B-Instruct-v0.3", "models/mistral_lora")
qwen, qwen_tok = load_lora("Qwen/Qwen2.5-7B-Instruct", "models/qwen_lora") 

2025-11-04 11:24:09.197795: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762255449.220475     146 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762255449.227700     146 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading models/phi3_lora...


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
import os; os._exit(00)

In [7]:
import pandas as pd
from tqdm import tqdm
test_df = pd.read_pickle("/kaggle/working/checkpoint_splits_20251103_211458.pkl")['test']

models = [
    (phi, phi_tok, "Phi"),
    (mistral, mistral_tok, "Mistral"),
    (qwen, qwen_tok, "Qwen")
]

def jury_score(row):
    scores = []
    for model, tok, name in models:  # ← Use global `models`
        prompt = f"""You are an expert at the {row['bloom_level']} level.
Prompt: {row['prompt']}
Response: {row['response']}
Score 1–5. JSON: {{"score": int, "rationale": str}}"""
        inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=100, temperature=0.3)
        resp = tok.decode(out[0], skip_special_tokens=True)
        try:
            import re, json
            j = re.search(r'\{.*\}', resp).group()
            data = json.loads(j)
            scores.append(int(data.get('score', 0)))
        except:
            scores.append(0)
    return round(sum(scores)/len(scores)) if scores else 0

jury_scores = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Jury Evaluation"):
    jury = jury_score(row)
    jury_scores.append(jury)

# Compute final metric
rho = test_df['essay_score'].corr(pd.Series(jury_scores), method='spearman')
print(f"FINAL JURY Spearman ρ: {rho:.3f}")

Jury Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Jury Evaluation:   1%|          | 1/100 [00:01<03:13,  1.96s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Jury Evaluation:   2%|▏         | 2/100 [00:14<13:14,  8.11s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The follow

FINAL JURY Spearman ρ: nan



/usr/local/lib/python3.11/dist-packages/pandas/core/nanops.py:1632: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  return spearmanr(a, b)[0]


In [8]:
import re, json

# Pick first row
row = test_df.iloc[0]
print("PROMPT:")
print(row['prompt'])
print("\nRESPONSE:")
print(row['response'][:500] + "..." if len(row['response']) > 500 else row['response'])
print(f"\nTRUE SCORE: {row['essay_score']}")
print(f"BLOOM LEVEL: {row['bloom_level']}")

model, tok = phi, phi_tok  # or mistral, qwen

prompt = f"""You are an expert at the {row['bloom_level']} level of Bloom's Taxonomy.
Prompt: {row['prompt']}
Response: {row['response']}
Score this response from 1 to 5 and provide a brief rationale.
Respond in JSON: {{"score": int, "rationale": str}}"""

inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to("cuda")

with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=100,
        temperature=0.1,
        do_sample=False
    )

raw_output = tok.decode(out[0], skip_special_tokens=True)
print("\nRAW MODEL OUTPUT:")
print(raw_output)

def extract_score(text):
    try:
        j = re.search(r'\{.*\}', text, re.DOTALL).group()
        data = json.loads(j)
        return int(data.get('score', 0)), data.get('rationale', '')
    except:
        return 0, "FAILED"

score, rationale = extract_score(raw_output)
print(f"\nEXTRACTED SCORE: {score}")
print(f"RATIONALE: {rationale}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


PROMPT:
Write a story about a time when you, or someone you know, was patient.

RESPONSE:
What exactly is patience? Patience means to wait calmly for something. One of the times that I had to be patient was when I wanted my ears pierced. I tryed everything for four years, begging, pleading, even cleaning, but it still didn't work. My mom says I have to be @NUM1 before I can get my ears pierced. So instead of annoying her, I sat back and stayed patient. By doing that, I got my mom to change the age to twelve. So all I have to do is wait until soccer is finished and find a place and sh...

TRUE SCORE: 20
BLOOM LEVEL: Create

RAW MODEL OUTPUT:
You are an expert at the Create level of Bloom's Taxonomy.
Prompt: Write a story about a time when you, or someone you know, was patient.
Response: What exactly is patience? Patience means to wait calmly for something. One of the times that I had to be patient was when I wanted my ears pierced. I tryed everything for four years, begging, pleading, e

In [11]:
row = test_df.iloc[0]
model, tok = phi, phi_tok  # Try Phi first

messages = [
    {"role": "system", "content": f"You are an expert evaluator at the {row['bloom_level']} level of Bloom's Taxonomy."},
    {"role": "user", "content": f"Prompt: {row['prompt']}\n\nStudent Response: {row['response']}\n\nScore this response from 1 to 5 and provide a brief rationale. Respond in JSON only: {{\"score\": int, \"rationale\": str}}"}
]

# Apply chat template
inputs = tok.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

# Generate
with torch.no_grad():
    out = model.generate(
        inputs,
        max_new_tokens=100,
        temperature=0.1,
        do_sample=False
    )

# Full raw output
raw = tok.decode(out[0], skip_special_tokens=True)
print("=== RAW OUTPUT ===")
print(raw)
print("=== END RAW ===")

# Try to extract
try:
    json_part = raw.split("<|assistant|>")[-1] if "<|assistant|>" in raw else raw
    print("\nJSON PART:")
    print(json_part)
    j = re.search(r'\{.*\}', json_part, re.DOTALL).group()
    data = json.loads(j)
    print(f"\nPARSED: score={data.get('score')}, rationale={data.get('rationale')}")
except Exception as e:
    print(f"\nPARSE FAILED: {e}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== RAW OUTPUT ===
You are an expert evaluator at the Create level of Bloom's Taxonomy. Prompt: Write a story about a time when you, or someone you know, was patient.

Student Response: What exactly is patience? Patience means to wait calmly for something. One of the times that I had to be patient was when I wanted my ears pierced. I tryed everything for four years, begging, pleading, even cleaning, but it still didn't work. My mom says I have to be @NUM1 before I can get my ears pierced. So instead of annoying her, I sat back and stayed patient. By doing that, I got my mom to change the age to twelve. So all I have to do is wait until soccer is finished and find a place and she'll take me. Patience really helps in many different ways and I encourage you to try being patient when you want something, because it will all fall into place.

Score this response from 1 to 5 and provide a brief rationale. Respond in JSON only: {"score": int, "rationale": str} {"score": 5, "rationale": "The re

In [19]:
import pandas as pd
df = pd.read_csv('/kaggle/input/asap-aes/training_set_rel3.tsv', sep='\t', encoding='ISO-8859-1')
print(df.groupby('essay_set')['domain1_score'].agg(['min', 'max']))

           min  max
essay_set          
1            2   12
2            1    6
3            0    3
4            0    3
5            0    4
6            0    4
7            2   24
8           10   60


In [24]:
# FULL FINAL CODE — RUN THIS
import pandas as pd
import torch
import re
import json
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load test data
test_df = pd.read_pickle("/kaggle/working/checkpoint_splits_20251103_211458.pkl")['test']

# Load 3 fine-tuned models
def load_lora(base_id, lora_path):
    print(f"Loading {lora_path}...")
    base = AutoModelForCausalLM.from_pretrained(
        base_id, load_in_4bit=True, device_map="auto", torch_dtype=torch.bfloat16
    )
    model = PeftModel.from_pretrained(base, lora_path)
    tokenizer = AutoTokenizer.from_pretrained(base_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# phi, phi_tok = load_lora("microsoft/Phi-3-mini-4k-instruct", "models/phi3_lora/checkpoint_300")
# mistral, mistral_tok = load_lora("mistralai/Mistral-7B-Instruct-v0.3", "models/mistral_lora/checkpoint_300")
# qwen, qwen_tok = load_lora("Qwen/Qwen2.5-7B-Instruct", "models/qwen_lora/checkpoint_300")

models = [(phi, phi_tok, "Phi"), (mistral, mistral_tok, "Mistral"), (qwen, qwen_tok, "Qwen")]

# Extract score from model output
def extract_score(text):
    try:
        j = re.findall(r'\{.*\}', text)[-1]
        data = json.loads(j)
        return int(data.get("score", 0))
    except:
        return 0

# Jury scoring
def jury_score(row):
    scores = []
    for model, tok, name in models:
        prompt = f"""<|system|>You are an expert evaluator at the {row['bloom_level']} level of Bloom's Taxonomy.<|end|>
<|user|>Prompt: {row['prompt']}\n\nStudent Response: {row['response']}\n\nScore this response from 1 to 5. Respond in JSON only: {{"score": int, "rationale": str}}<|end|>
<|assistant|>"""
        inputs = tok(prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            out = model.generate(**inputs, max_new_tokens=80, temperature=0.0, do_sample=False)
        resp = tok.decode(out[0], skip_special_tokens=True)
        score = extract_score(resp)
        scores.append(score)
    return round(sum(scores)/len(scores)) if scores else 0

# Run jury
jury_scores = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Jury Evaluation"):
    jury_scores.append(jury_score(row))

# Normalize 1–5 → ASAP scale per essay_set
scale_map = {
    1: (2, 12), 2: (1, 6), 3: (0, 3), 4: (0, 3),
    5: (0, 4), 6: (0, 4), 7: (2, 24), 8: (10, 60)
}

def normalize(jury_1to5, essay_set):
    min_s, max_s = scale_map[essay_set]
    return round(min_s + (jury_1to5 - 1) * (max_s - min_s) / 4)

jury_normalized = [
    normalize(j, r['essay_set']) for j, (_, r) in zip(jury_scores, test_df.iterrows())
]

# Compute final Spearman ρ
rho = test_df['essay_score'].corr(pd.Series(jury_normalized), method='spearman')
print(f"\nFINAL JURY Spearman ρ: {rho:.3f}")

# Save results
results = pd.DataFrame({
    'essay_set': test_df['essay_set'],
    'true_score': test_df['essay_score'],
    'jury_1to5': jury_scores,
    'jury_normalized': jury_normalized
})
results.to_csv("/kaggle/working/jury_final_normalized.csv", index=False)
print("Results saved to jury_final_normalized.csv")

Jury Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Jury Evaluation:   1%|          | 1/100 [00:14<23:51, 14.46s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be 


FINAL JURY Spearman ρ: -0.431
Results saved to jury_final_normalized.csv


In [13]:
def extract_score(text):
    try:
        # Find ALL JSON blocks
        json_blocks = re.findall(r'\{.*?\}', text, re.DOTALL)
        if not json_blocks:
            return 0, "No JSON"
        # Take the LAST one (model's actual answer)
        j = json_blocks[-1]
        data = json.loads(j)
        return int(data.get('score', 0)), data.get('rationale', '')
    except Exception as e:
        return 0, f"Parse error: {e}"

def jury_score(row):
    scores = []
    for model, tok, name in models:
        messages = [
            {"role": "system", "content": f"You are an expert evaluator at the {row['bloom_level']} level of Bloom's Taxonomy."},
            {"role": "user", "content": f"Prompt: {row['prompt']}\n\nStudent Response: {row['response']}\n\nScore this response from 1 to 5 and provide a brief rationale. Respond in JSON only: {{\"score\": int, \"rationale\": str}}"}
        ]
        inputs = tok.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to("cuda")

        with torch.no_grad():
            out = model.generate(
                inputs,
                max_new_tokens=100,
                temperature=0.1,
                do_sample=False
            )
        resp = tok.decode(out[0], skip_special_tokens=True)
        
        score, _ = extract_score(resp)
        scores.append(score)
    
    return round(sum(scores)/len(scores)) if scores else 0

row = test_df.iloc[0]
score = jury_score(row)
print(f"Jury score: {score}")  # Should be 5

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Jury score: 2


In [27]:
def jury_score(row):
    scores = []
    for model, tok, name in models:
        messages = [
            {"role": "system", "content": f"You are an expert evaluator at the {row['bloom_level']} level."},
            {"role": "user", "content": f"""Prompt: {row['prompt']}\n\nResponse: {row['response']}\n\nScore 1–5. JSON only."""}
        ]
        
        # Use chat template + force JSON
        inputs = tok.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True
        ).to("cuda")
        
        # Force JSON output
        generate_kwargs = dict(
            inputs,
            max_new_tokens=100,
            temperature=0.0,
            do_sample=False,
            response_format={"type": "json_object"}  # ← Critical
        )
        
        with torch.no_grad():
            out = model.generate(**generate_kwargs)
        
        resp = tok.decode(out[0], skip_special_tokens=True)
        
        # Extract JSON
        try:
            j = re.search(r'\{.*\}', resp, re.DOTALL).group()
            data = json.loads(j)
            score = int(data.get("score", 0))
        except:
            score = 0
        scores.append(score)
    
    return round(sum(scores)/len(scores)) if scores else 0

In [28]:
# ------------------------------------------------------------------
# 1. Imports (run once – they are already imported in your notebook)
# ------------------------------------------------------------------
import pandas as pd
import torch
import re
import json
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

# ------------------------------------------------------------------
# 2. Load test data (first 2 rows only)
# ------------------------------------------------------------------
test_df = pd.read_pickle("/kaggle/working/checkpoint_splits_20251103_211458.pkl")['test']
test_df = test_df.head(2).reset_index(drop=True)      # <-- ONLY FIRST 2 ROWS
print("=== TEST ROWS ===")
print(test_df[['essay_set', 'essay_score', 'bloom_level']].to_string(index=False))
print("\n")

# ------------------------------------------------------------------
# 3. Load the three fine-tuned adapters
# ------------------------------------------------------------------
def load_lora(base_id, lora_path):
    print(f"Loading {lora_path} …")
    base = AutoModelForCausalLM.from_pretrained(
        base_id, load_in_4bit=True, device_map="auto", torch_dtype=torch.bfloat16
    )
    model = PeftModel.from_pretrained(base, lora_path)
    tokenizer = AutoTokenizer.from_pretrained(base_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# phi,   phi_tok   = load_lora("microsoft/Phi-3-mini-4k-instruct",
#                             "models/phi3_lora/checkpoint_300")
# mistral, mistral_tok = load_lora("mistralai/Mistral-7B-Instruct-v0.3",
#                                  "models/mistral_lora/checkpoint_300")
# qwen,   qwen_tok   = load_lora("Qwen/Qwen2.5-7B-Instruct",
#                                "models/qwen_lora/checkpoint_300")

models = [(phi, phi_tok, "Phi"),
          (mistral, mistral_tok, "Mistral"),
          (qwen, qwen_tok, "Qwen")]

# ------------------------------------------------------------------
# 4. JSON extraction (robust)
# ------------------------------------------------------------------
def extract_score(raw_text):
    """Return (score, raw_json_string) – score=0 on failure."""
    try:
        # Grab the last {...} block
        matches = re.findall(r'\{.*\}', raw_text, re.DOTALL)
        if not matches:
            return 0, ""
        j = matches[-1]
        data = json.loads(j)
        return int(data.get("score", 0)), j
    except Exception as e:
        return 0, f"PARSE ERROR: {e}"

# ------------------------------------------------------------------
# 5. Jury scoring (one row)
# ------------------------------------------------------------------
def jury_score(row):
    per_model = {}
    for model, tok, name in models:
        messages = [
            {"role": "system",
             "content": f"You are an expert evaluator at the {row['bloom_level']} level of Bloom's Taxonomy."},
            {"role": "user",
             "content": f"Prompt: {row['prompt']}\n\nStudent Response: {row['response']}\n\n"
                        f"Score this response from 1 to 5 and provide a brief rationale. "
                        f"Respond in JSON only: {{\"score\": int, \"rationale\": str}}"}
        ]

        inputs = tok.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True
        ).to("cuda")

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=120,
                temperature=0.0,
                do_sample=False
            )
        resp = tok.decode(out[0], skip_special_tokens=True)

        score, json_blob = extract_score(resp)
        per_model[name] = {"score": score, "json": json_blob, "raw": resp[-400:]}  # last 400 chars

    # Aggregate
    scores = [v["score"] for v in per_model.values()]
    jury_1to5 = round(sum(scores) / len(scores)) if scores else 0
    return jury_1to5, per_model

# ------------------------------------------------------------------
# 6. Run on first 2 rows
# ------------------------------------------------------------------
scale_map = {1: (2,12), 2:(1,6), 3:(0,3), 4:(0,3),
             5:(0,4), 6:(0,4), 7:(2,24), 8:(10,60)}

def normalize(jury_1to5, essay_set):
    mn, mx = scale_map.get(essay_set, (0,12))
    return round(mn + (jury_1to5-1)*(mx-mn)/4)

print("\n=== JURY TEST (first 2 rows) ===")
for idx, row in test_df.iterrows():
    jury_1to5, details = jury_score(row)
    jury_norm = normalize(jury_1to5, row['essay_set'])

    print(f"\n--- ROW {idx} (Set {row['essay_set']}) ---")
    print(f"True score : {row['essay_score']}")
    print(f"Jury 1-5   : {jury_1to5}")
    print(f"Jury norm  : {jury_norm}")

    for name, info in details.items():
        print(f"\n  [{name}] score = {info['score']}")
        print(f"  JSON   : {info['json'][:200]}{'...' if len(info['json'])>200 else ''}")
        # Uncomment next line if you want the *full* raw output
        print(f"  RAW    : {info['raw']}")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


=== TEST ROWS ===
 essay_set  essay_score bloom_level
         7           20      Create
         2            2      Create



=== JURY TEST (first 2 rows) ===


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- ROW 0 (Set 7) ---
True score : 20
Jury 1-5   : 0
Jury norm  : -4

  [Phi] score = 0
  JSON   : PARSE ERROR: Expecting value: line 1 column 11 (char 10)
  RAW    : de a brief rationale. Respond in JSON only: {"score": int, "rationale": str} {"score": 5, "rationale": "The response demonstrates a clear understanding of the concept of patience and provides a personal example of how patience was exercised. The student also explains the importance of patience and encourages others to try it. The response is well-written and effectively communicates the message."}

  [Mistral] score = 0
  JSON   : PARSE ERROR: Expecting value: line 1 column 11 (char 10)
  RAW    : , I got my mom to change the age to twelve. So all I have to do is wait until soccer is finished and find a place and she'll take me. Patience really helps in many different ways and I encourage you to try being patient when you want something, because it will all fall into place.

Score this response from 1 to 5 and provide a 

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



--- ROW 1 (Set 2) ---
True score : 2
Jury 1-5   : 0
Jury norm  : 0

  [Phi] score = 0
  JSON   : PARSE ERROR: Expecting value: line 1 column 11 (char 10)
  RAW    :  understanding of the prompt and provides some valid points about the importance of censorship in libraries. However, the response lacks organization and clarity, making it difficult to follow. The student's arguments are not well-developed and are often presented in a confusing manner. The response would benefit from a more structured approach and a clearer explanation of the points being made."}

  [Mistral] score = 0
  JSON   : PARSE ERROR: Expecting value: line 1 column 11 (char 10)
  RAW    : ake a lot mof monoey.     So that why i think they should not take stuff off because that is how we are loseing all hour money.The people are useing the that they would for movies or books and stuff they are useing for drugs and other stuff that can get them sent to jail and stuff

Score this response from 1 to 5 and provide a br

In [34]:
train_df = pd.read_pickle("/kaggle/working/checkpoint_splits_20251103_211458.pkl")['train']
train_df.head(2)

,prompt,response,essay_score,essay_set,bloom_level,label_score,rationale
55,The writers had to write a persuasive essay re...,I think that libraries should not remove mater...,3,2,Create,5,The question asks students to produce a new pi...
487,"Write a story about a time when you, or someon...",Patientce is hard for example when you walk in...,10,7,Create,5,The question asks the user to generate a new n...


In [35]:
train_df['label_score'].value_counts()

label_score
5    300
Name: count, dtype: int64

In [36]:
print("label_score distribution:")
print(train_df['label_score'].value_counts())
print("\nessay_score distribution:")
print(train_df['essay_score'].describe())

label_score distribution:
label_score
5    300
Name: count, dtype: int64

essay_score distribution:
count    300.000000
mean       7.320000
std        9.301577
min        0.000000
25%        2.000000
50%        3.000000
75%        8.250000
max       45.000000
Name: essay_score, dtype: float64


## Final

In [37]:
# FINAL CODE — RUN THIS
import pandas as pd
import torch
import re
import json
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load test data
test_df = pd.read_pickle("/kaggle/working/checkpoint_splits_20251103_211458.pkl")['test']

# Load 3 fine-tuned models
def load_lora(base_id, lora_path):
    print(f"Loading {lora_path}...")
    base = AutoModelForCausalLM.from_pretrained(
        base_id, load_in_4bit=True, device_map="auto", torch_dtype=torch.bfloat16
    )
    model = PeftModel.from_pretrained(base, lora_path)
    tokenizer = AutoTokenizer.from_pretrained(base_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# phi, phi_tok = load_lora("microsoft/Phi-3-mini-4k-instruct", "models/phi3_lora/checkpoint_300")
# mistral, mistral_tok = load_lora("mistralai/Mistral-7B-Instruct-v0.3", "models/mistral_lora/checkpoint_300")
# qwen, qwen_tok = load_lora("Qwen/Qwen2.5-7B-Instruct", "models/qwen_lora/checkpoint_300")

models = [(phi, phi_tok, "Phi"), (mistral, mistral_tok, "Mistral"), (qwen, qwen_tok, "Qwen")]

# Extract score from model output
def extract_score(text):
    try:
        j = re.findall(r'\{.*\}', text, re.DOTALL)[-1]
        data = json.loads(j)
        return int(data.get("score", 0))
    except:
        return 0

# Jury scoring
def jury_score(row):
    scores = []
    for model, tok, name in models:
        messages = [
            {"role": "system", "content": f"You are an expert evaluator at the {row['bloom_level']} level of Bloom's Taxonomy."},
            {"role": "user", "content": f"Prompt: {row['prompt']}\n\nStudent Response: {row['response']}\n\nScore this response from 1 to 5 and provide a brief rationale. Respond in JSON only: {{\"score\": int, \"rationale\": str}}"}
        ]
        inputs = tok.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True
        ).to("cuda")

        with torch.no_grad():
            out = model.generate(
                **inputs,
                max_new_tokens=100,
                temperature=0.0,
                do_sample=False
            )
        resp = tok.decode(out[0], skip_special_tokens=True)
        score = extract_score(resp)
        scores.append(score)
    return round(sum(scores)/len(scores)) if scores else 0

# Run jury
jury_scores = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Jury Evaluation"):
    jury_scores.append(jury_score(row))

# Normalize 1–5 → ASAP scale per essay_set (CORRECTED)
scale_map = {
    1: (2, 12),   # Set 1
    2: (1, 6),    # Set 2
    3: (0, 3),    # Set 3
    4: (0, 3),    # Set 4
    5: (0, 4),    # Set 5
    6: (0, 4),    # Set 6
    7: (2, 24),   # Set 7
    8: (10, 60)   # Set 8
}

def normalize(jury_1to5, essay_set):
    min_s, max_s = scale_map.get(essay_set, (0, 12))
    return round(min_s + (jury_1to5 - 1) * (max_s - min_s) / 4)

jury_normalized = [
    normalize(j, r['essay_set']) for j, (_, r) in zip(jury_scores, test_df.iterrows())
]

# Compute final Spearman ρ
rho = test_df['essay_score'].corr(pd.Series(jury_normalized), method='spearman')
print(f"\nFINAL JURY Spearman ρ: {rho:.3f}")

# Save results
results = pd.DataFrame({
    'essay_set': test_df['essay_set'],
    'true_score': test_df['essay_score'],
    'jury_1to5': jury_scores,
    'jury_normalized': jury_normalized
})
results.to_csv("/kaggle/working/jury_final_normalized.csv", index=False)
print("Results saved to jury_final_normalized.csv")

Jury Evaluation:   0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Jury Evaluation:   1%|          | 1/100 [00:05<09:53,  5.99s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be 


FINAL JURY Spearman ρ: -0.431
Results saved to jury_final_normalized.csv


In [38]:
results

,essay_set,true_score,jury_1to5,jury_normalized
393,7,20,0,-4
309,2,2,0,0
3,7,17,0,-4
129,7,14,0,-4
404,4,0,0,-1
...,...,...,...,...
333,4,3,0,-1
85,7,8,0,-4
116,7,20,0,-4
427,2,3,0,0


In [40]:
# FINAL CODE — RUN THIS
import pandas as pd
import torch
import re
import json
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

test_df = pd.read_pickle("/kaggle/working/checkpoint_splits_20251103_211458.pkl")['test']

def load_lora(base_id, lora_path):
    base = AutoModelForCausalLM.from_pretrained(
        base_id, load_in_4bit=True, device_map="auto", torch_dtype=torch.bfloat16
    )
    model = PeftModel.from_pretrained(base, lora_path)
    tokenizer = AutoTokenizer.from_pretrained(base_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

# phi, phi_tok = load_lora("microsoft/Phi-3-mini-4k-instruct", "models/phi3_lora/checkpoint_300")
# mistral, mistral_tok = load_lora("mistralai/Mistral-7B-Instruct-v0.3", "models/mistral_lora/checkpoint_300")
# qwen, qwen_tok = load_lora("Qwen/Qwen2.5-7B-Instruct", "models/qwen_lora/checkpoint_300")

models = [(phi, phi_tok, "Phi"), (mistral, mistral_tok, "Mistral"), (qwen, qwen_tok, "Qwen")]

def jury_score(row):
    scores = []
    for model, tok, name in models:
        messages = [
            {"role": "system", "content": f"You are an expert evaluator at the {row['bloom_level']} level."},
            {"role": "user", "content": f"Prompt: {row['prompt']}\n\nResponse: {row['response']}\n\nScore 1–5. JSON only."}
        ]
        inputs = tok.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True
        ).to("cuda")

        with torch.no_grad():
            out = model.generate(
                input_ids=inputs,
                max_new_tokens=80,
                temperature=0.0,
                do_sample=False
            )
        resp = tok.decode(out[0], skip_special_tokens=True)
        try:
            j = re.search(r'\{.*\}', resp, re.DOTALL).group()
            data = json.loads(j)
            score = int(data.get("score", 0))
        except:
            score = 0
        scores.append(score)
    return round(sum(scores)/len(scores)) if scores else 0

jury_scores = [jury_score(row) for _, row in tqdm(test_df.iterrows(), total=100)]

# Normalize
scale_map = {1: (2,12), 2:(1,6), 3:(0,3), 4:(0,3), 5:(0,4), 6:(0,4), 7:(2,24), 8:(10,60)}
def normalize(j, s): 
    mn, mx = scale_map[s]
    return round(mn + (j-1)*(mx-mn)/4)

jury_normalized = [normalize(j, r['essay_set']) for j, (_, r) in zip(jury_scores, test_df.iterrows())]

rho = test_df['essay_score'].corr(pd.Series(jury_normalized), method='spearman')
print(f"FINAL ρ: {rho:.3f}")

results = pd.DataFrame({'true_score': test_df['essay_score'], 'jury_normalized': jury_normalized})
results.to_csv("jury_final.csv", index=False)


  0%|          | 0/100 [00:00<?, ?it/s]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

  1%|          | 1/100 [00:10<17:08, 10.39s/it]The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=i

FINAL ρ: -0.162


In [41]:
def jury_score(row):
    scores = []
    for model, tok, name in models:
        messages = [
            {"role": "system", "content": f"You are an expert evaluator at the {row['bloom_level']} level."},
            {"role": "user", "content": f"Prompt: {row['prompt']}\n\nResponse: {row['response']}\n\nScore 1–5. JSON only."}
        ]
        inputs = tok.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True
        ).to("cuda")

        # Force JSON
        generate_kwargs = dict(
            input_ids=inputs,
            max_new_tokens=80,
            temperature=0.0,
            do_sample=False,
            response_format={"type": "json_object"}
        )
        
        with torch.no_grad():
            out = model.generate(**generate_kwargs)
        
        resp = tok.decode(out[0], skip_special_tokens=True)
        
        try:
            j = re.search(r'\{.*\}', resp, re.DOTALL).group()
            data = json.loads(j)
            score = int(data.get("score", 0))
        except:
            score = 0
        scores.append(score)
    
    return round(sum(scores)/len(scores)) if scores else 0

In [50]:
# FINAL CODE — RUN THIS
import pandas as pd
import torch
import re
import json
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load test data
test_df = pd.read_pickle("/kaggle/working/checkpoint_splits_20251103_211458.pkl")['test']

# Load 3 fine-tuned models
def load_lora(base_id, lora_path):
    base = AutoModelForCausalLM.from_pretrained(
        base_id, load_in_4bit=True, device_map="auto", torch_dtype=torch.bfloat16
    )
    # Force load from checkpoint
    model = PeftModel.from_pretrained(base, lora_path, is_trainable=False)
    tokenizer = AutoTokenizer.from_pretrained(base_id)
    tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer

phi, phi_tok = load_lora("microsoft/Phi-3-mini-4k-instruct", "models/phi3_lora/checkpoint_300")
mistral, mistral_tok = load_lora("mistralai/Mistral-7B-Instruct-v0.3", "models/mistral_lora/checkpoint_300")
qwen, qwen_tok = load_lora("Qwen/Qwen2.5-7B-Instruct", "models/qwen_lora/checkpoint_300")

models = [(phi, phi_tok, "Phi"), (mistral, mistral_tok, "Mistral"), (qwen, qwen_tok, "Qwen")]

# Extract score from model output
def extract_score(text):
    try:
        j = re.findall(r'\{.*\}', text, re.DOTALL)[-1]
        data = json.loads(j)
        return int(data.get("score", 0))
    except:
        return 0

# Jury scoring
def jury_score(row):
    scores = []
    for model, tok, name in models:
        messages = [
            {"role": "system", "content": f"You are an expert evaluator at the {row['bloom_level']} level of Bloom's Taxonomy."},
            {"role": "user", "content": f"Prompt: {row['prompt']}\n\nStudent Response: {row['response']}\n\nScore this response from 1 to 5. Respond in JSON only: {{\"score\": int, \"rationale\": str}}"}
        ]
        inputs = tok.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True,
            tokenize=True,
            return_dict=True
        )["input_ids"].to("cuda")

        with torch.no_grad():
            out = model.generate(
                input_ids=inputs,
                max_new_tokens=100,
                temperature=0.0,
                do_sample=False
            )
        resp = tok.decode(out[0], skip_special_tokens=True)
        score = extract_score(resp)
        scores.append(score)
    return round(sum(scores)/len(scores)) if scores else 0

# Run jury
jury_scores = []
for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Jury Evaluation"):
    jury_scores.append(jury_score(row))

# Normalize 1–5 → ASAP scale per essay_set (CORRECTED)
scale_map = {
    1: (2, 12),   # Set 1
    2: (1, 6),    # Set 2
    3: (0, 3),    # Set 3
    4: (0, 3),    # Set 4
    5: (0, 4),    # Set 5
    6: (0, 4),    # Set 6
    7: (2, 24),   # Set 7
    8: (10, 60)   # Set 8
}

def normalize(jury_1to5, essay_set):
    min_s, max_s = scale_map.get(essay_set, (0, 12))
    return round(min_s + (jury_1to5 - 1) * (max_s - min_s) / 4)

jury_normalized = [
    normalize(j, r['essay_set']) for j, (_, r) in zip(jury_scores, test_df.iterrows())
]

# Compute final Spearman ρ
rho = test_df['essay_score'].corr(pd.Series(jury_normalized), method='spearman')
print(f"\nFINAL JURY Spearman ρ: {rho:.3f}")

# Save results
results = pd.DataFrame({
    'essay_set': test_df['essay_set'],
    'true_score': test_df['essay_score'],
    'jury_1to5': jury_scores,
    'jury_normalized': jury_normalized
})
results.to_csv("/kaggle/working/jury_final_normalized.csv", index=False)
print("Results saved to jury_final_normalized.csv")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: Can't find 'adapter_config.json' at 'models/phi3_lora/checkpoint_300'

In [54]:
from IPython.display import FileLink
FileLink(r'/kaggle/working/kaggle_working_directory.zip')

/kaggle/working/kaggle_working_directory.zip

In [56]:
!zip -r -s 2g /kaggle/working/working_split.zip /kaggle/working/kaggle_working_directory.zip

  adding: kaggle/working/kaggle_working_directory.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (stored 0%)


In [80]:
gc.collect()
torch.cuda.empty_cache()

In [75]:
"""
Reduce Test Set to 50 Essays
=============================
This creates a smaller test set for faster evaluation.
"""

import pandas as pd

print("="*80)
print("REDUCING TEST SET TO 50 ESSAYS")
print("="*80)

# Load original data
original_path = '/kaggle/working/checkpoint_splits_20251103_211458.pkl'
print(f"\nLoading: {original_path}")

data = pd.read_pickle(original_path)
test_df = data['test']

print(f"Original test set size: {len(test_df)} essays")

# Sample 50 essays (stratified by bloom_level if possible)
if 'bloom_level' in test_df.columns:
    # Try to get balanced sample
    bloom_counts = test_df['bloom_level'].value_counts()
    print(f"\nBloom level distribution in original:")
    print(bloom_counts)
    
    # Sample proportionally
    n_per_level = min(25, len(test_df) // len(bloom_counts))
    sampled = []
    
    for level in test_df['bloom_level'].unique():
        level_df = test_df[test_df['bloom_level'] == level]
        sample_size = min(n_per_level, len(level_df))
        sampled.append(level_df.sample(sample_size, random_state=42))
    
    test_small = pd.concat(sampled).sample(frac=1, random_state=42)  # Shuffle
    
    # If we got less than 50, fill up randomly
    if len(test_small) < 50:
        remaining = test_df[~test_df.index.isin(test_small.index)]
        additional = remaining.sample(min(50 - len(test_small), len(remaining)), random_state=42)
        test_small = pd.concat([test_small, additional])
else:
    # Simple random sample
    test_small = test_df.sample(min(50, len(test_df)), random_state=42)

print(f"\nReduced test set size: {len(test_small)} essays")

if 'bloom_level' in test_small.columns:
    print(f"\nBloom level distribution in reduced set:")
    print(test_small['bloom_level'].value_counts())

# Save
data['test'] = test_small
output_path = '/kaggle/working/checkpoint_splits_SMALL.pkl'
pd.to_pickle(data, output_path)

print(f"\n✓ Saved to: {output_path}")

# Estimate time
essays = len(test_small)
models = 3
seconds_per_essay = 60
total_minutes = (essays * models * seconds_per_essay) / 60
total_hours = total_minutes / 60

print(f"\n" + "="*80)
print("TIME ESTIMATE")
print("="*80)
print(f"  Essays: {essays}")
print(f"  Models: {models}")
print(f"  Speed: ~{seconds_per_essay}s per essay")
print(f"  Total: ~{total_minutes:.0f} minutes (~{total_hours:.1f} hours)")
print(f"\n  With 8 hours GPU time remaining, you have {8 - total_hours:.1f} hours buffer")
print("="*80)

REDUCING TEST SET TO 50 ESSAYS

Loading: /kaggle/working/checkpoint_splits_20251103_211458.pkl
Original test set size: 100 essays

Bloom level distribution in original:
bloom_level
Analyze    55
Create     45
Name: count, dtype: int64

Reduced test set size: 50 essays

Bloom level distribution in reduced set:
bloom_level
Create     25
Analyze    25
Name: count, dtype: int64

✓ Saved to: /kaggle/working/checkpoint_splits_SMALL.pkl

TIME ESTIMATE
  Essays: 50
  Models: 3
  Speed: ~60s per essay
  Total: ~150 minutes (~2.5 hours)

  With 8 hours GPU time remaining, you have 5.5 hours buffer


In [81]:
"""
BloomLLM-Jury: MEMORY-EFFICIENT Evaluation Script
==================================================
This version loads ONE model at a time to avoid GPU OOM errors.

Key difference from original:
- Load model 1 → Evaluate all essays → Unload → Clear memory
- Load model 2 → Evaluate all essays → Unload → Clear memory
- Load model 3 → Evaluate all essays → Unload → Clear memory
- Then aggregate results

This is slower but works with limited GPU memory.
"""

import pandas as pd
import numpy as np
import re
import json
import torch
import gc
from tqdm import tqdm
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from scipy.stats import spearmanr, mannwhitneyu, pearsonr
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================

CONFIG = {
    'models': [
        {
            'name': 'Phi',
            'base_id': 'microsoft/Phi-3-mini-4k-instruct',
            'lora_path': '/kaggle/working/models/phi3_lora'
        },
        {
            'name': 'Mistral',
            'base_id': 'mistralai/Mistral-7B-Instruct-v0.3',
            'lora_path': '/kaggle/working/models/mistral_lora'
        },
        {
            'name': 'Qwen',
            'base_id': 'Qwen/Qwen2.5-7B-Instruct',
            'lora_path': '/kaggle/working/models/qwen_lora'
        }
    ],
    'test_data_path': '/kaggle/working/checkpoint_splits_SMALL.pkl',  # UPDATED to use smaller set
    'output_dir': '/kaggle/working',
    'generation_params': {
        'max_new_tokens': 250,
        'do_sample': False,  # Greedy decoding for consistency
    },
}

# ASAP dataset score ranges per essay set (from official documentation)
ASAP_SCORE_RANGES = {
    1: (1, 6),    # Argumentative
    2: (1, 6),    # Argumentative
    3: (0, 3),    # Source-Dependent
    4: (0, 3),    # Source-Dependent
    5: (0, 4),    # Source-Dependent
    6: (0, 4),    # Source-Dependent
    7: (0, 3),    # Narrative
    8: (1, 6),    # Narrative
}

# ============================================================================
# MODEL LOADING (ONE AT A TIME)
# ============================================================================

def load_single_model(base_id, lora_path):
    """Load a single model with proper memory management."""
    print(f"\n{'='*80}")
    print(f"Loading {base_id}...")
    print(f"{'='*80}")
    
    # Clear any existing models
    torch.cuda.empty_cache()
    gc.collect()
    
    # Proper 4-bit quantization config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    
    # Load base model
    base_model = AutoModelForCausalLM.from_pretrained(
        base_id,
        quantization_config=bnb_config,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
    )
    
    print(f"  ✓ Base model loaded")
    
    # Apply LoRA
    model = PeftModel.from_pretrained(base_model, lora_path)
    print(f"  ✓ LoRA adapter applied from {lora_path}")
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(base_id, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    print(f"  ✓ Tokenizer loaded")
    
    # Print memory usage
    if torch.cuda.is_available():
        mem_allocated = torch.cuda.memory_allocated(0) / 1e9
        mem_reserved = torch.cuda.memory_reserved(0) / 1e9
        print(f"  📊 GPU Memory: {mem_allocated:.2f}GB allocated, {mem_reserved:.2f}GB reserved")
    
    return model, tokenizer


def unload_model(model, tokenizer):
    """Properly unload model and free memory."""
    del model
    del tokenizer
    torch.cuda.empty_cache()
    gc.collect()
    print(f"  ✓ Model unloaded, memory cleared")


# ============================================================================
# SCORE EXTRACTION
# ============================================================================

def extract_score_robust(text):
    """Extract score and rationale from model output."""
    original_text = text
    
    # Clean up text
    assistant_markers = ['<|assistant|>', '[/INST]', 'assistant:', 'Assistant:']
    for marker in assistant_markers:
        if marker in text:
            text = text.split(marker)[-1]
    
    text = text.strip()
    
    # Method 1: JSON
    json_pattern = r'\{[^}]*"score"\s*:\s*\d+[^}]*\}'
    json_matches = re.findall(json_pattern, text, re.DOTALL | re.IGNORECASE)
    
    for match in json_matches:
        try:
            data = json.loads(match)
            score = int(data.get('score', 0))
            
            if 1 <= score <= 5:
                rationale = data.get('rationale', data.get('reason', data.get('explanation', '')))
                return score, rationale, "JSON"
        except:
            continue
    
    # Method 2: Patterns
    score_patterns = [
        r'"score"\s*:\s*(\d+)',
        r'score\s*:\s*(\d+)',
        r'Score\s*:\s*(\d+)',
        r'\*\*Score\*\*\s*:\s*(\d+)',
    ]
    
    for pattern in score_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            score = int(match.group(1))
            if 1 <= score <= 5:
                return score, "", "Pattern"
    
    # Method 3: Ratio
    ratio_patterns = [r'\b([1-5])\s*/\s*5\b', r'\b([1-5])\s+out\s+of\s+5\b']
    for pattern in ratio_patterns:
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            return int(match.group(1)), "", "Ratio"
    
    # Method 4: Any digit 1-5
    digit_matches = re.findall(r'\b([1-5])\b', text)
    if digit_matches:
        return int(digit_matches[0]), "", "Weak"
    
    # Failed
    return 3, "Extraction failed", "Failed"


# ============================================================================
# INFERENCE
# ============================================================================

def construct_prompt(row):
    """Construct evaluation prompt."""
    system_msg = f"You are an expert educational evaluator specializing in Bloom's Taxonomy. You evaluate student responses for {row['bloom_level']}-level cognitive skills."
    
    user_msg = f"""Evaluate this student response for {row['bloom_level']}-level thinking according to Bloom's Taxonomy.

**Prompt**: {row['prompt']}

**Student Response**: {row['response']}

Provide your evaluation in this EXACT JSON format:
{{"score": <integer 1-5>, "rationale": "<brief explanation>"}}

**Scoring Guide**:
- 5: Excellent - Fully demonstrates {row['bloom_level']}-level thinking
- 4: Good - Strong demonstration with minor gaps
- 3: Adequate - Meets basic requirements
- 2: Weak - Minimal {row['bloom_level']}-level thinking
- 1: Poor - Does not demonstrate {row['bloom_level']}-level thinking

Respond ONLY with the JSON object."""

    return [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_msg}
    ]


def evaluate_with_model(model, tokenizer, row, config):
    """Evaluate a single essay with a model."""
    messages = construct_prompt(row)
    
    try:
        inputs = tokenizer.apply_chat_template(
            messages,
            return_tensors="pt",
            add_generation_prompt=True,
            padding=True,
            truncation=True
        ).to(model.device)
    except Exception as e:
        return {
            'score': 3,
            'rationale': f"Tokenization error: {e}",
            'extraction_method': 'Error',
            'raw_output': ''
        }
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_new_tokens=config['generation_params']['max_new_tokens'],
            do_sample=config['generation_params']['do_sample'],
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=False,  # Fix for DynamicCache error
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    score, rationale, method = extract_score_robust(response)
    
    return {
        'score': score,
        'rationale': rationale,
        'extraction_method': method,
        'raw_output': response
    }


# ============================================================================
# MAIN EVALUATION (ONE MODEL AT A TIME)
# ============================================================================

def evaluate_jury_memory_efficient(test_df, model_configs, config):
    """
    Evaluate jury by loading ONE model at a time.
    This avoids GPU OOM errors.
    """
    print("\n" + "="*80)
    print("MEMORY-EFFICIENT JURY EVALUATION")
    print("="*80)
    print("\nStrategy: Load → Evaluate → Unload → Repeat for each model")
    print("This is slower but works with limited GPU memory.\n")
    
    # Storage for all results
    all_model_results = {}  # {model_name: [scores for each essay]}
    all_rationales = {}     # {model_name: [rationales for each essay]}
    all_extraction_methods = {}  # {model_name: [methods for each essay]}
    
    # Evaluate with each model
    for model_config in model_configs:
        model_name = model_config['name']
        
        print(f"\n{'='*80}")
        print(f"EVALUATING WITH {model_name}")
        print(f"{'='*80}")
        
        # Load this model
        model, tokenizer = load_single_model(
            model_config['base_id'],
            model_config['lora_path']
        )
        
        # Evaluate all essays with this model
        model_scores = []
        model_rationales = []
        model_methods = []
        
        print(f"\nEvaluating {len(test_df)} essays...")
        
        for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc=f"{model_name}"):
            result = evaluate_with_model(model, tokenizer, row, config)
            
            model_scores.append(result['score'])
            model_rationales.append(result['rationale'])
            model_methods.append(result['extraction_method'])
        
        # Store results
        all_model_results[model_name] = model_scores
        all_rationales[model_name] = model_rationales
        all_extraction_methods[model_name] = model_methods
        
        # Print summary
        extraction_success = sum(1 for m in model_methods if m in ['JSON', 'Pattern']) / len(model_methods) * 100
        print(f"\n  📊 {model_name} Summary:")
        print(f"     Extraction success: {extraction_success:.1f}%")
        print(f"     Mean score: {np.mean(model_scores):.2f} ± {np.std(model_scores):.2f}")
        print(f"     Score distribution: {Counter(model_scores)}")
        
        # Unload this model before loading next
        unload_model(model, tokenizer)
    
    # Aggregate jury results
    print(f"\n{'='*80}")
    print("AGGREGATING JURY DECISIONS")
    print(f"{'='*80}")
    
    jury_scores = []
    for i in range(len(test_df)):
        essay_scores = [all_model_results[name][i] for name in all_model_results.keys()]
        jury_score = round(np.mean(essay_scores))
        jury_scores.append(jury_score)
    
    # Compile results
    results = {
        'individual_scores': all_model_results,
        'jury_scores': jury_scores,
        'rationales': [],
        'extraction_methods': all_extraction_methods,
        'true_scores': test_df['essay_score'].tolist(),
        'bloom_levels': test_df['bloom_level'].tolist(),
        'essay_sets': test_df['essay_set'].tolist() if 'essay_set' in test_df.columns else [None] * len(test_df),
    }
    
    # Format rationales
    for i in range(len(test_df)):
        essay_rationales = {name: all_rationales[name][i] for name in all_rationales.keys()}
        results['rationales'].append(essay_rationales)
    
    return results


# ============================================================================
# ANALYSIS FUNCTIONS (same as before)
# ============================================================================

def calculate_inter_jury_agreement(results, model_names):
    """Calculate Cohen's kappa between models."""
    print("\n" + "="*80)
    print("INTER-JURY AGREEMENT")
    print("="*80)
    
    pairwise_kappas = {}
    for i, name1 in enumerate(model_names):
        for j, name2 in enumerate(model_names):
            if i < j:
                kappa = cohen_kappa_score(
                    results['individual_scores'][name1],
                    results['individual_scores'][name2],
                    weights='linear'
                )
                pairwise_kappas[f"{name1}-{name2}"] = kappa
                print(f"  {name1:10s} vs {name2:10s}: κ = {kappa:.3f}")
    
    mean_kappa = np.mean(list(pairwise_kappas.values()))
    print(f"\n  📊 Mean κ: {mean_kappa:.3f}")
    
    return {'mean_kappa': mean_kappa, 'pairwise_kappas': pairwise_kappas}


def analyze_bloom_discrimination(results):
    """Test Bloom level discrimination."""
    print("\n" + "="*80)
    print("BLOOM'S LEVEL DISCRIMINATION")
    print("="*80)
    
    df = pd.DataFrame({
        'jury_score': results['jury_scores'],
        'bloom_level': results['bloom_levels']
    })
    
    print("\nScore Distribution by Bloom Level:")
    print(df.groupby('bloom_level')['jury_score'].describe())
    
    bloom_levels = df['bloom_level'].unique()
    
    if len(bloom_levels) == 2:
        level1, level2 = bloom_levels
        scores1 = df[df['bloom_level'] == level1]['jury_score']
        scores2 = df[df['bloom_level'] == level2]['jury_score']
        
        statistic, p_value = mannwhitneyu(scores1, scores2, alternative='two-sided')
        
        print(f"\n📊 Mann-Whitney U Test: {level1} vs {level2}")
        print(f"  p-value: {p_value:.4f}")
        print(f"  {'Significant!' if p_value < 0.05 else 'Not significant'}")
        
        return {'p_value': p_value, 'significant': p_value < 0.05}
    
    return None


def calculate_asap_correlation(results):
    """Calculate normalized correlation with ASAP."""
    print("\n" + "="*80)
    print("ASAP CORRELATION")
    print("="*80)
    
    jury_norm = [(s - 1) / 4 for s in results['jury_scores']]
    asap_norm = []
    
    for score, essay_set in zip(results['true_scores'], results['essay_sets']):
        if essay_set and essay_set in ASAP_SCORE_RANGES:
            min_s, max_s = ASAP_SCORE_RANGES[essay_set]
            norm = (score - min_s) / (max_s - min_s)
            asap_norm.append(max(0, min(1, norm)))
        else:
            asap_norm.append(score / max(results['true_scores']))
    
    rho, p = spearmanr(asap_norm, jury_norm)
    
    print(f"\n📊 Spearman ρ: {rho:.3f} (p={p:.4f})")
    print(f"  {'Significant!' if p < 0.05 else 'Not significant'}")
    
    return {'rho': rho, 'p_value': p}


def export_results(results, model_names, output_dir):
    """Export results to CSV."""
    print("\n" + "="*80)
    print("EXPORTING RESULTS")
    print("="*80)
    
    # Main results
    results_df = pd.DataFrame({
        'true_score': results['true_scores'],
        'jury_score': results['jury_scores'],
        'bloom_level': results['bloom_levels'],
        **{f'{name}_score': results['individual_scores'][name] for name in model_names}
    })
    
    results_df.to_csv(f"{output_dir}/jury_results_complete.csv", index=False)
    print(f"  ✓ Saved: jury_results_complete.csv")


# ============================================================================
# MAIN
# ============================================================================

def main():
    """Main execution."""
    print("\n" + "="*80)
    print("BloomLLM-Jury: Memory-Efficient Evaluation")
    print("="*80)
    
    # Load test data
    print("\n[1/5] Loading test data...")
    test_df = pd.read_pickle(CONFIG['test_data_path'])['test']
    print(f"  ✓ Loaded {len(test_df)} essays")
    
    # Evaluate jury (one model at a time)
    print("\n[2/5] Evaluating jury...")
    results = evaluate_jury_memory_efficient(test_df, CONFIG['models'], CONFIG)
    
    # Get model names
    model_names = [m['name'] for m in CONFIG['models']]
    
    # Calculate metrics
    print("\n[3/5] Calculating metrics...")
    agreement = calculate_inter_jury_agreement(results, model_names)
    bloom = analyze_bloom_discrimination(results)
    asap = calculate_asap_correlation(results)
    
    # Export
    print("\n[4/5] Exporting results...")
    export_results(results, model_names, CONFIG['output_dir'])
    
    # Summary
    print("\n" + "="*80)
    print("SUMMARY")
    print("="*80)
    print(f"  Inter-jury κ: {agreement['mean_kappa']:.3f}")
    if bloom:
        print(f"  Bloom p-value: {bloom['p_value']:.4f}")
    print(f"  ASAP ρ: {asap['rho']:.3f}")
    print("\n✓ Evaluation complete!")


if __name__ == "__main__":
    main()


BloomLLM-Jury: Memory-Efficient Evaluation

[1/5] Loading test data...
  ✓ Loaded 50 essays

[2/5] Evaluating jury...

MEMORY-EFFICIENT JURY EVALUATION

Strategy: Load → Evaluate → Unload → Repeat for each model
This is slower but works with limited GPU memory.


EVALUATING WITH Phi

Loading microsoft/Phi-3-mini-4k-instruct...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  ✓ Base model loaded
  ✓ LoRA adapter applied from /kaggle/working/models/phi3_lora
  ✓ Tokenizer loaded
  📊 GPU Memory: 15.12GB allocated, 15.53GB reserved

Evaluating 50 essays...


Phi: 100%|██████████| 50/50 [1:16:05<00:00, 91.32s/it] 



  📊 Phi Summary:
     Extraction success: 100.0%
     Mean score: 3.44 ± 0.92
     Score distribution: Counter({4: 31, 2: 13, 3: 4, 5: 2})
  ✓ Model unloaded, memory cleared

EVALUATING WITH Mistral

Loading mistralai/Mistral-7B-Instruct-v0.3...


ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

In [82]:
!rm -f /kaggle/working/kaggle_working_directory.zip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [83]:
!rm -f /kaggle/working/working_split.z*

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [84]:
!du -sh /kaggle/working/

166M	/kaggle/working/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [85]:
!tar -czf backup.tar.gz -C /kaggle/working .

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tar: .: file changed as we read it


In [87]:
!ls -lh /kaggle/working/backup.tar.gz

-rw-r--r-- 1 root root 129M Nov  4 17:34 /kaggle/working/backup.tar.gz


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [88]:
!tar -tzf /kaggle/working/backup.tar.gz | head -30

./
./bcls.csv
./data/
./data/blooms_taxonomy_cognitive_levels.csv
./wandb/
./wandb/run-20251103_223127-5ke5qob7/
./wandb/run-20251103_223127-5ke5qob7/logs/
./wandb/run-20251103_223127-5ke5qob7/logs/debug.log
./wandb/run-20251103_223127-5ke5qob7/logs/debug-internal.log
./wandb/run-20251103_223127-5ke5qob7/files/
./wandb/run-20251103_223127-5ke5qob7/files/wandb-summary.json
./wandb/run-20251103_223127-5ke5qob7/files/output.log
./wandb/run-20251103_223127-5ke5qob7/files/wandb-metadata.json
./wandb/run-20251103_223127-5ke5qob7/files/requirements.txt
./wandb/run-20251103_223127-5ke5qob7/files/config.yaml
./wandb/run-20251103_223127-5ke5qob7/run-5ke5qob7.wandb
./wandb/run-20251103_225045-dluzy0il/
./wandb/run-20251103_225045-dluzy0il/run-dluzy0il.wandb
./wandb/run-20251103_225045-dluzy0il/logs/
./wandb/run-20251103_225045-dluzy0il/logs/debug.log
./wandb/run-20251103_225045-dluzy0il/logs/debug-internal.log
./wandb/run-20251103_225045-dluzy0il/files/
./wandb/run-20251103_225045-dluzy0il/files/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tar: stdout: write error


In [89]:
!tar -tzf /kaggle/working/backup.tar.gz | wc -l

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


397


In [90]:
!pip install -q PyDrive

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [92]:
# Method 1: Using gdown (reverse - upload instead of download)
# This won't work either, so use Method 2

# Method 2: Use Google Drive REST API directly
!pip install -q google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

# Build Drive API
drive_service = build('drive', 'v3')

# Create folder
folder_metadata = {
    'name': 'BloomLLM_Kaggle_Backup',
    'mimeType': 'application/vnd.google-apps.folder'
}
folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
folder_id = folder.get('id')

print(f"✓ Created folder ID: {folder_id}")

# Upload file
print("\nUploading backup.tar.gz...")
file_metadata = {
    'name': 'backup.tar.gz',
    'parents': [folder_id]
}
media = MediaFileUpload('/kaggle/working/backup.tar.gz', resumable=True)
file = drive_service.files().create(
    body=file_metadata,
    media_body=media,
    fields='id'
).execute()

print(f"\n✅ UPLOAD COMPLETE!")
print(f"📁 View at: https://drive.google.com/drive/folders/{folder_id}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


✓ Created folder ID: 1kwQNheHg_PFVMvr1oAGSmmgmJX5dT82E

Uploading backup.tar.gz...

✅ UPLOAD COMPLETE!
📁 View at: https://drive.google.com/drive/folders/1kwQNheHg_PFVMvr1oAGSmmgmJX5dT82E
